In [ ]:
import cv2
import numpy as np
import tensorflow as tf

IMAGE_PATH = 'cat.jpg'
LAYER_NAME = 'block5_conv3'
CAT_CLASS_INDEX = 281

img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(224, 224))
img = tf.keras.preprocessing.image.img_to_array(img)

model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)

grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img]))
    loss = predictions[:, CAT_CLASS_INDEX]

output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))

cam = np.ones(output.shape[0: 2], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, :, i]

cam = cv2.resize(cam.numpy(), (224, 224))
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / (cam.max() - cam.min())

cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)

cv2.imwrite('cam.png', output_image)

503324672/553467096 [==========================>...] - ETA: 1s

In [10]:
!wget https://upload.wikimedia.org/wikipedia/commons/1/1a/Arthur%2C_the_cat.jpg > cat.jpg

--2020-09-01 11:01:25--  https://upload.wikimedia.org/wikipedia/commons/1/1a/Arthur%2C_the_cat.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208, 2620:0:862:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2636769 (2.5M) [image/jpeg]
Saving to: ‘Arthur,_the_cat.jpg’

Arthur,_the_cat.jpg 100%[===================>]   2.51M  --.-KB/s    in 0.1s    

2020-09-01 11:01:25 (25.0 MB/s) - ‘Arthur,_the_cat.jpg’ saved [2636769/2636769]



In [9]:
curl -O https://www.python.org/static/apple-touch-icon-144x144-precomposed.png

SyntaxError: ignored